In [1]:
####### 데이터 불러오고 서브셋 만듦
from itertools import chain
from collections import defaultdict
from torch.utils.data import Subset
from torchvision import datasets


def subset_sampler(dataset, classes, max_len):
    target_idx = defaultdict(list) # 각 클래스의 인덱스를 저장하는 dict
    # dataset.train_labels=각 이미지에 해당하는 클래스(label)(0~9)의 정수 배열
    for idx, label in enumerate(dataset.train_labels):
        target_idx[int(label)].append(idx)

    # 각 클래스별로 max_len만큼의 인덱스를 추출하여 하나의 리스트로 만듦
    indices = list(
        chain.from_iterable(
            [target_idx[idx][:max_len] for idx in range(len(classes))]
        )
    )
    return Subset(dataset, indices)


train_dataset = datasets.FashionMNIST(root="../datasets", download=True, train=True)
test_dataset  = datasets.FashionMNIST(root="../datasets", download=True, train=False)

# train dataset에 포함된 클래스
classes = train_dataset.classes
# train dataset의 클래스와 클래스id가 매핑된 값
class_to_idx = train_dataset.class_to_idx

# 각 클래스별로 최대 1000개의 샘플을 포함하는 훈련 데이터 서브셋 만듦
subset_train_dataset = subset_sampler(
    dataset = train_dataset, classes = train_dataset.classes, max_len = 1000
)

# 각 클래스별로 최대 100개의 샘플을 포함하는 테스트 데이터 서브셋 만듦
subset_test_dataset = subset_sampler(
    dataset = test_dataset, classes = test_dataset.classes, max_len = 100
)

/opt/homebrew/lib/python3.11/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [2]:
####### 이미지 전처리
import torch
from torchvision import transforms
from transformers import AutoImageProcessor


image_processor = AutoImageProcessor.from_pretrained(
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k"
)

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(
            size=(
                image_processor.size["height"],
                image_processor.size["width"]
            )
        ),
        transforms.Lambda(
            lambda x: torch.cat([x, x, x], 0)
        ),
        transforms.Normalize(
            mean = image_processor.image_mean,
            std  = image_processor.image_std
        )
    ]
)

/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
####### 모델 구조에 맞는 형태로 변환 데이터로더 적용
from torch.utils.data import DataLoader

# swin transformer모델은 Tensor 형식이 아닌 딕셔너리 형식의 데이터를 입력으로 받음
# swin transformer모델의 입력: {"pixel_values":pixel_values, "labels":labels}

def collator(data, transform):
    images, labels = zip(*data)
    # pixel_values = (배치크기, 채널수, 이미지높이, 이미지너비)
    pixel_values = torch.stack([transform(image) for image in images])
    # labels = [클래스 색인값]
    labels       = torch.tensor([label for label in labels])
    return {"pixel_values": pixel_values, "labels": labels}


train_dataloader = DataLoader(
    subset_train_dataset, # (PIL이미지, label)
    batch_size = 32,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)
valid_dataloader = DataLoader(
    subset_test_dataset,
    batch_size = 4,
    shuffle    = True,
    collate_fn = lambda x: collator(x, transform),
    drop_last  = True
)

In [4]:
####### 10.13 사전학습된 스윈 트랜스포머 모델
from transformers import SwinForImageClassification


model = SwinForImageClassification.from_pretrained(
    pretrained_model_name_or_path = "microsoft/swin-tiny-patch4-window7-224",
    # 224*224 이미지, 4*4 크기의 이미지 패치, 최종 7*7개의 로컬 윈도
    num_labels                    = len(train_dataset.classes),
    id2label                      = {idx: label for label, idx in train_dataset.class_to_idx.items()},
    label2id                      = train_dataset.class_to_idx,
    ignore_mismatched_sizes       = True
)

for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                if sssub_name == "projection":
                    print("│  │  └", sssub_name, sssub_module)
                else:
                    print("│  │  └", sssub_name)
# swin은 cls 토큰 대신 마지막 출력 토큰을 평균하여 사용. 이 평균 작업을 pooler가 함

/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


swin
└ embeddings
│  └ patch_embeddings
│  │  └ projection Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
│  └ norm
│  └ dropout
└ encoder
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
└ layernorm
└ pooler
classifier


In [5]:
####### 10.14 패치 임베딩 모듈
batch = next(iter(train_dataloader))
print("이미지 차원 :", batch["pixel_values"].shape)

# 패치 임베딩 (224*224*3이미지 - 4*4커널크기, 4간격 -> 56*56(=3136)개 패치(채널=96))
patch_emb_output, shape = model.swin.embeddings.patch_embeddings(batch["pixel_values"])
print("모듈:", model.swin.embeddings.patch_embeddings)
print("패치 임베딩 차원 :", patch_emb_output.shape)

이미지 차원 : torch.Size([32, 3, 224, 224])
모듈: SwinPatchEmbeddings(
  (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
)
패치 임베딩 차원 : torch.Size([32, 3136, 96])


/opt/homebrew/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [6]:
####### 10.15 스윈 트랜스포머 블록
# stage 1의 스윈 트랜스포머 블록 구조 - blocks(스윈트랜스포머블록), downsample(patch merging)
for main_name, main_module in model.swin.encoder.layers[0].named_children():
    print(main_name) 
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│ └", ssub_name)

blocks
└ 0
│ └ layernorm_before
│ └ attention
│ └ drop_path
│ └ layernorm_after
│ └ intermediate
│ └ output
└ 1
│ └ layernorm_before
│ └ attention
│ └ drop_path
│ └ layernorm_after
│ └ intermediate
│ └ output
downsample
└ reduction
└ norm


In [7]:
####### 10.16 SwinLayer(스윈 트랜스포머 블록) 구조
# head가 1개인 경우 (linear에서 채널이 1/head가 되는데 여기선 변화 X)
print(model.swin.encoder.layers[0].blocks[0])

SwinLayer(
  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  (attention): SwinAttention(
    (self): SwinSelfAttention(
      (query): Linear(in_features=96, out_features=96, bias=True)
      (key): Linear(in_features=96, out_features=96, bias=True)
      (value): Linear(in_features=96, out_features=96, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (output): SwinSelfOutput(
      (dense): Linear(in_features=96, out_features=96, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (drop_path): SwinDropPath(p=0.1)
  (layernorm_after): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  (intermediate): SwinIntermediate(
    (dense): Linear(in_features=96, out_features=384, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): SwinOutput(
    (dense): Linear(in_features=384, out_features=96, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
)


- layernorm_before : 첫 번째 계층 정규화
- attention : W-MSA(SW-MSA)
- layer_norm_after : 두 번째 계층 정규화
- intermediate, output : MLP

In [8]:
####### 10.17 W-MSA, SW-MSA 모듈
print("패치 임베딩 차원 :", patch_emb_output.shape)

W_MSA  = model.swin.encoder.layers[0].blocks[0]
SW_MSA = model.swin.encoder.layers[0].blocks[1]

W_MSA_output  = W_MSA(patch_emb_output, W_MSA.input_resolution)[0]
SW_MSA_output = SW_MSA(W_MSA_output, SW_MSA.input_resolution)[0]

print("W-MSA 결과 차원 :", W_MSA_output.shape)
print("SW-MSA 결과 차원 :", SW_MSA_output.shape)

패치 임베딩 차원 : torch.Size([32, 3136, 96])
W-MSA 결과 차원 : torch.Size([32, 3136, 96])
SW-MSA 결과 차원 : torch.Size([32, 3136, 96])


In [9]:
####### 10.18 패치 병합 - 패치를 절반 크기로 분할하는 reduction, 계층 정규화
# 1. 2*2 그룹으로 나눠 채널기준으로 병합 : 32*3136(56*56)*96 -> 32*784(28*28)*384(96*4)
# 2. 차원을 절반으로 축소 : 32*784*384 -> 32*784*192
patch_merge = model.swin.encoder.layers[0].downsample
print("patch_merge 모듈 :", patch_merge)

# patch_merge로 해상도 1/4배, 채널 2배
output = patch_merge(SW_MSA_output, patch_merge.input_resolution)
print("patch_merge 결과 차원 :", output.shape)

# 이렇게 마지막 스테이지4까지 통과하면 [N, 49(7*7), 768]이 됨
# 이 특징맵이 풀링 계층을 통과해 클래스를 예측함

patch_merge 모듈 : SwinPatchMerging(
  (reduction): Linear(in_features=384, out_features=192, bias=False)
  (norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
)
patch_merge 결과 차원 : torch.Size([32, 784, 192])
